In [4]:
import pandas as pd
import numpy as np

from ckiptagger import WS, POS
from tqdm.notebook import tqdm

In [112]:
DATA_PATH = '../../../data/'
df_train = pd.read_csv(DATA_PATH + 'news_clf/news_clustering_train.tsv', sep='\t')
df_test = pd.read_csv(DATA_PATH + 'news_clf/news_clustering_test.tsv', sep='\t')

df_train.info(), print('\n'), df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   1800 non-null   int64 
 1   class   1800 non-null   object
 2   title   1800 non-null   object
dtypes: int64(1), object(2)
memory usage: 42.3+ KB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   600 non-null    int64 
 1   class   600 non-null    object
 2   title   600 non-null    object
dtypes: int64(1), object(2)
memory usage: 14.2+ KB


(None, None, None)

In [102]:
df_train.head()

,index,class,title
0,0,體育,亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列
1,1,體育,9輪4球本土射手僅次武磊 黃紫昌要搶最強U23頭銜
2,2,體育,如果今年勇士奪冠，下賽季詹姆斯何去何從？
3,3,體育,超級替補！科斯塔本賽季替補出場貢獻7次助攻
4,4,體育,騎士6天里發生了啥？從首輪搶七到次輪3-0猛龍


In [101]:
df_train.tail()

,index,class,title
1795,1795,遊戲,LOL：麻辣香鍋韓服Rank不合成打野刀？電刀巨魔新套路連勝中
1796,1796,遊戲,穩住，我們能贏！因為我們擁有這種強大的力量
1797,1797,遊戲,騰訊是怎樣毀掉《鬥戰神》這款可能成神的作品的？
1798,1798,遊戲,LOL你不知道的黑科技打法！
1799,1799,遊戲,遊戲拳皇各代的劇情都在講些什麼？


In [104]:
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

test_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
test_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

# 斷詞 + POS

In [ ]:
ws = WS(DATA_PATH+'ckip_data/')
pos = POS(DATA_PATH+'ckip_data/')

In [161]:
train_title_cuts = {}
for index, title in tqdm(train_titles.items()):
    # YOUR CODE HERE
    word_s = ws([title], sentence_segmentation=True)
    word_p = pos([title])
    # END YOUR CODE
    train_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [163]:
test_title_cuts = {}
for index, title in tqdm(test_titles.items()):
    # YOUR CODE HERE
    word_s = ws([title], sentence_segmentation=True)
    word_p = pos([title])
    # END YOUR CODE
    test_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [164]:
train_title_cuts[120]

[('國腳', 'Nc'),
 ('張呈棟', 'Na'),
 ('：', 'Nb'),
 ('從', 'VJ'),
 ('沒', 'Nf'),
 ('想', 'COLONCATEGORY'),
 ('過', 'D'),
 ('自己', 'D'),
 ('會', 'VE'),
 ('出', 'Di'),
 ('一', 'P'),
 ('本', 'Nh'),
 ('書', 'D')]

---
# Bag of Words (BOW)

In [233]:
word2index = {}
index2word = {}

# 產生字與index對應的關係
# YOUR CODE HERE

all_words = [word for pairs in train_title_cuts.values() for word, _ in pairs]
unique_words = list(set(all_words))
print(f'所有單字：{len(all_words)}, 不重複單字：{len(unique_words)}')

for index, word in enumerate(unique_words):
    word2index[word] = index
    index2word[index] = word
    
# END YOUR CODE

所有單字：25333, 不重複單字：6662


In [235]:
word2index['溫暖'], index2word[3988]

(3988, '溫暖')

In [238]:
def get_bow_vector(pairs, word2index):
    # YOUR CODE HERE
    
    vector = np.zeros(len(unique_words))
    for word, _ in pairs:
        vector[word2index[word]] += 1
        
    # END YOUR CODE
    return vector

In [239]:
get_bow_vector(train_title_cuts[120], word2index)

array([0., 0., 0., ..., 0., 0., 0.])

# 排除較無意義的詞性

In [243]:
pos_analysis = {}
for _, pairs in train_title_cuts.items():
    for word, flag in pairs:
        if flag not in pos_analysis:
            pos_analysis[flag] = set()
        pos_analysis[flag].add(word)

for flag, words in pos_analysis.items():
    print(flag, ':', list(words)[:10])
    print('----------------------------------------------------------------------------')

Nc : ['堅決', '筆', '第114', '縣區', '所在', '東方', '招待', '遵義市', '重磅', '越來越']
----------------------------------------------------------------------------
Na : ['報價', '新作', '險勝', '走低', '額度', '武磊', '東方', '做生意', '報關', '訣竅']
----------------------------------------------------------------------------
VC : ['3000億', '快速', '差點', '優惠券', '擺', '榮譽', '速遞', '猛', '外出', '十']
----------------------------------------------------------------------------
VD : ['貢獻', '還', '這', '徵信', '追債', '全方位', '巴中市', '都', '什麼', '發酵劑']
----------------------------------------------------------------------------
COLONCATEGORY : ['不應該', '多', '黃金', '還', '這', '日乒', '戶', '投票成', '剛', ' ']
----------------------------------------------------------------------------
PAUSECATEGORY : ['是', '，', '棉紡', '和', '炒股', '價格', '您', '年', '數據', ' ']
----------------------------------------------------------------------------
Neu : ['本質', '人和', 'iPad', '調整', '十', '歲', '億航', '聖堂山', '特價', 'pro7']
-------------------------------------------------------

|         Type        |     Description    |
|:-------------------:|:------------------:|
| A                   | 非謂形容詞         |
| Caa                 | 對等連接詞         |
| Cab                 | 連接詞，如：等等   |
| Cba                 | 連接詞，如：的話   |
| Cbb                 | 關聯連接詞         |
| D                   | 副詞               |
| Da                  | 數量副詞           |
| Dfa                 | 動詞前程度副詞     |
| Dfb                 | 動詞後程度副詞     |
| Di                  | 時態標記           |
| Dk                  | 句副詞             |
| DM                  | 定量式             |
| I                   | 感嘆詞             |
| Na                  | 普通名詞           |
| Nb                  | 專有名詞           |
| Nc                  | 地方詞             |
| Ncd                 | 位置詞             |
| Nd                  | 時間詞             |
| Nep                 | 指代定詞           |
| Neqa                | 數量定詞           |
| Neqb                | 後置數量定詞       |
| Nes                 | 特指定詞           |
| Neu                 | 數詞定詞           |
| Nf                  | 量詞               |
| Ng                  | 後置詞             |
| Nh                  | 代名詞             |
| Nv                  | 名物化動詞         |
| P                   | 介詞               |
| T                   | 語助詞             |
| VA                  | 動作不及物動詞     |
| VAC                 | 動作使動動詞       |
| VB                  | 動作類及物動詞     |
| VC                  | 動作及物動詞       |
| VCL                 | 動作接地方賓語動詞 |
| VD                  | 雙賓動詞           |
| VF                  | 動作謂賓動詞       |
| VE                  | 動作句賓動詞       |
| VG                  | 分類動詞           |
| VH                  | 狀態不及物動詞     |
| VHC                 | 狀態使動動詞       |
| VI                  | 狀態類及物動詞     |
| VJ                  | 狀態及物動詞       |
| VK                  | 狀態句賓動詞       |
| VL                  | 狀態謂賓動詞       |
| V_2                 | 有                 |
|                     |                    |
| DE                  | 的之得地           |
| SHI                 | 是                 |
| FW                  | 外文               |
|                     |                    |
| COLONCATEGORY       | 冒號               |
| COMMACATEGORY       | 逗號               |
| DASHCATEGORY        | 破折號             |
| DOTCATEGORY         | 點號               |
| ETCCATEGORY         | 刪節號             |
| EXCLAMATIONCATEGORY | 驚嘆號             |
| PARENTHESISCATEGORY | 括號               |
| PAUSECATEGORY       | 頓號               |
| PERIODCATEGORY      | 句號               |
| QUESTIONCATEGORY    | 問號               |
| SEMICOLONCATEGORY   | 分號               |
| SPCHANGECATEGORY    | 雙直線             |
| WHITESPACE          | 空白               |

In [244]:
def get_bow_vector_with_selection(pairs, word2index):
    excluded_flags = [
        # 根據以上列舉出來的文字以及詞性表，請列出想要排除的詞性
        # YOUR CODE HERE
        'Dfb', 'DOTCATEGORY', 'Da', 'DE'
        # END YOUR CODE
    ]
    vector = np.zeros(len(word2index))
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            vector[word2index[word]] += 1
    return vector

# Cosine Similarity

In [248]:
def cosine_similarity(bow1, bow2):
    # YOUR CODE HERE
    
    v1 = bow1 / (np.sqrt(np.sum(bow1**2)) + 1e-8)
    v2 = bow2 / (np.sqrt(np.sum(bow2**2)) + 1e-8)
    similarity = np.dot(v1, v2)
    
    # END YOUR CODE
    return similarity

In [249]:
bow1 = get_bow_vector(train_title_cuts[100], word2index)
bow2 = get_bow_vector(train_title_cuts[130], word2index)
cosine_similarity(bow1, bow2)

0.08703882746415753

In [250]:
train_title_cuts[100]

[('山東', 'Na'),
 ('魯能', 'Ncd'),
 ('有沒有', 'Nc'),
 ('可能', 'D'),
 ('拿到', 'V_2'),
 ('今年', 'D'),
 ('的', 'V_2'),
 ('中', 'D'),
 ('超', 'D'),
 ('冠軍', 'VC'),
 ('？', 'P')]

In [251]:
train_title_cuts[130]

[('NBA', 'FW'),
 ('和', 'FW'),
 ('CBA', 'FW'),
 ('差距', 'Caa'),
 ('在', 'FW'),
 ('哪裡', 'FW'),
 ('？', 'FW'),
 ('6', 'VH'),
 ('張', 'Na'),
 ('圖', 'P'),
 ('一目瞭然', 'Nep'),
 ('！', 'Ncd')]

---
# Group mean vector: 訓練

In [253]:
group_vectors = {news_class: [] for news_class in all_news_class}
group_vectors

{'體育': [], '財經': [], '科技': [], '旅遊': [], '農業': [], '遊戲': []}

In [260]:
for index, pairs in sorted(train_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    news_class = train_classes[index]
    group_vectors[news_class].append(vector)

group_vectors['體育'][:5]

[array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.])]

In [261]:
group_mean_vector = {}

for news_class, vectors in group_vectors.items():
    group_mean_vector[news_class] = np.mean(vectors, axis=0)
    
group_mean_vector

{'體育': array([0.00333333, 0.        , 0.        , ..., 0.00666667, 0.02666667,
        0.        ]),
 '財經': array([0.        , 0.00333333, 0.        , ..., 0.00333333, 0.04      ,
        0.        ]),
 '科技': array([0.        , 0.        , 0.        , ..., 0.01333333, 0.01666667,
        0.        ]),
 '旅遊': array([0.        , 0.        , 0.        , ..., 0.01333333, 0.01      ,
        0.00333333]),
 '農業': array([0.        , 0.        , 0.00333333, ..., 0.01      , 0.05      ,
        0.        ]),
 '遊戲': array([0.        , 0.        , 0.        , ..., 0.01      , 0.02666667,
        0.        ])}

# Group mean vector: 測試

In [262]:
classification = {news_class: [] for news_class in all_news_class}

for index, pairs in sorted(test_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    if np.sum(np.square(vector)) == 0:
        continue

    max_val = -2.0
    max_class = None
    for news_class, ref_vector in group_mean_vector.items():
        val = cosine_similarity(ref_vector, vector)
        if val > max_val:
            max_class = news_class
            max_val = val

    classification[max_class].append(index)

In [263]:
from collections import Counter

for group, ids in classification.items():
    counter = Counter([test_classes[id] for id in ids])
    print('predict', group, ': ', counter)

predict 體育 :  Counter({'體育': 53, '財經': 18, '農業': 13, '科技': 10, '旅遊': 10, '遊戲': 8})
predict 財經 :  Counter({'財經': 28, '科技': 9, '農業': 7, '旅遊': 6, '體育': 5, '遊戲': 2})
predict 科技 :  Counter({'科技': 59, '旅遊': 27, '財經': 25, '農業': 22, '遊戲': 21, '體育': 20})
predict 旅遊 :  Counter({'旅遊': 44, '財經': 11, '農業': 11, '遊戲': 8, '體育': 4, '科技': 3})
predict 農業 :  Counter({'農業': 43, '科技': 13, '遊戲': 10, '體育': 9, '財經': 8, '旅遊': 7})
predict 遊戲 :  Counter({'遊戲': 51, '財經': 10, '體育': 8, '旅遊': 6, '科技': 5, '農業': 4})
